In [ ]:
!pip install cantera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 36.7 MB/s eta 0:00:00


In [ ]:
## Please compare with the attached .txt file in the prompt.
# This version is the complete one

import time
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import cantera as ct
from functools import lru_cache

#
# 0. Global constants and cycle settings
#
Ta, Pa = 233, 20000         # ISA sea level
cp_air, gamma_air = 1005.0, 1.400   # constant ideal‑gas properties

n = 0.92
TT   = 1500      # K
eta_c = n # compressor
eta_t = 0.9 # turbine
fan_pr, eta_fan = 1.80, 0.9 # fan
BPR   = 5.0 # need
M0    = 0.79
eta_comb = 0.98
LHV   = 43e6 # J kg⁻¹
D = 2.39 # inlet diameter


# 1.  Parse thermo databases

fuel_gas_template = ct.Solution("Mechanism1.yaml")
air_gas           = ct.Solution("air.yaml")
air_N2_O2         = {"O2": 1, "N2": 3.76}# 79/21 vol


# 2. Combustor solver


#@lru_cache(maxsize=128)
def combustor_dodecane(T_in: float, P_in: float):

    gas = fuel_gas_template

    def _res(phi):
        gas.set_equivalence_ratio(phi, {"NC12H26": 1}, air_N2_O2)
        gas.TP = T_in, P_in
        gas.equilibrate("HP")
        return gas.T - TT

    # Brent bracket
    phi_min, phi_max = 0.1, 1.5
    f_min, f_max = _res(phi_min), _res(phi_max)

    if f_min * f_max > 0.0:
        phi = phi_min if abs(f_min) < abs(f_max) else phi_max
    else:
        phi = opt.brentq(_res, phi_min, phi_max, xtol=1e-4, maxiter=15)

    #h_out     = gas.enthalpy_mass
    #cp_out    = gas.cp_mass
    #gamma_out = gas.cp_mass / gas.cv_mass
    h_out     = cp_air * TT
    cp_out    = cp_air
    gamma_out = gamma_air
    return phi, h_out, cp_out, gamma_out


# 3.  Turbofan cycle

def turbofan_cantera(rp: float):

    R_air = (gamma_air - 1.0) / gamma_air * cp_air

    # 3.1 Freestream
    a0  = np.sqrt(gamma_air * R_air * Ta)
    V0  = M0 * a0
    Tt0 = Ta * (1.0 + 0.5 * (gamma_air - 1.0) * M0**2)
    rho = Pa / (R_air * Ta)
    m_in = rho * np.pi * (D / 2) ** 2 * V0

    # 3.2 Compressor
    Tt2 = Tt0 * (1.0 + (rp ** ((gamma_air - 1.0) / gamma_air) - 1.0) / eta_c)
    w_c = cp_air * (Tt2 - Tt0)

    # 3.3 Combustor (Cantera)
    P3 = Pa * rp
    phi, h4, cp4, gamma4 = combustor_dodecane(Tt2, P3)

    # 3.3b Fuel–air ratio
    h2 = cp_air * Tt2
    f  = (h4 - h2) / (eta_comb * LHV - h4)

    # 3.4 Turbine
    Tt4 = TT
    Tt5 = Tt4 - w_c / (eta_t * cp4)
    rp_t = (Tt4 / Tt5) ** (gamma4 / (gamma4 - 1.0))

    # 3.5 Core nozzle (perfect expansion)
    V9 = np.sqrt(2.0 * cp4 * Tt5 * (1.0 - (Pa / (P3 / rp_t)) ** ((gamma4 - 1.0) / gamma4)))

    # 3.6 Bypass stream
    Ttf = Tt0 * (1.0 + (fan_pr ** ((gamma_air - 1.0) / gamma_air) - 1.0) / eta_fan)
    Vf  = np.sqrt(2.0 * cp_air * Ttf * (1.0 - (Pa / (Pa * fan_pr)) ** ((gamma_air - 1.0) / gamma_air)))

    # 3.7 Performance
    Fs   = (1.0 + f) * V9 + BPR * Vf - (1.0 + BPR) * V0
    TSFC = f / Fs
    return Fs, TSFC, phi, m_in, f


# 4. Objective function for optimizer

w_F, w_S = 1.0, 5.0e4  # weights: favor high thrust, penalize TSFC heavily

def objective(rp: float) -> float:
    Fs, S, _, _, _ = turbofan_cantera(rp)
    return -(w_F * Fs - w_S * S)


# 5. Solve for optimum OPR

if __name__ == "__main__":
    t0 = time.perf_counter()

    # Use minimise_scalar with the bounds by Brent
    result = opt.minimize_scalar(
        objective,
        bounds=(10.0, 50.0),
        method="bounded",
        options={"xatol": 0.1}  # tolerance on rp
    )

    elapsed = time.perf_counter() - t0

    rp_best = result.x
    Fs_best, TSFC_best, phi_best, m_dot, f_final = turbofan_cantera(rp_best)

    print("Ideal cycle + Cantera combustor")
    print(f"Optimisation finished in {elapsed:.2f} s, iterations: {result.nit}")
    print(f"Best overall pressure ratio : {rp_best:6.2f}")
    print(f"Specific thrust Fs          : {Fs_best:8.1f}  N·s/kg_air")
    print(f"TSFC                        : {TSFC_best:10.5f}  kg/N/s")
    print(f"Equivalence ratio φ         : {phi_best:6.3f}")  #
    print(f"Mass flow compressor         : {m_dot / (1+BPR):6.3f} kg/s")
    print(f"Fuel mass flow        : {f_final:6.3f} kg/s")  #

    # Quick diagnostic plot around the optimum (optional)
    #rps = np.linspace(5.0, 55.0, 20)
    #Js  = np.array([objective(r) for r in rps])

    #plt.figure()
    #plt.plot(rps, Js, label="Objective J")
    #plt.plot(rp_best, objective(rp_best), "o", label=f"Best rp ≈ {rp_best:.2f}")
    #plt.xlabel("Overall pressure ratio (OPR)")
    #plt.ylabel("J = -F_s + 5e4·TSFC")
    #plt.title("OPR optimisation — ideal cycle + FAST Cantera combustor")
    #plt.grid(ls="--", lw=0.5)
    #plt.legend()
    #plt.tight_layout()
    #plt.show()


Ideal cycle + Cantera combustor
Optimisation finished in 0.16 s, iterations: 10
Best overall pressure ratio :  14.76
Specific thrust Fs          :   1074.2  N·s/kg_air
TSFC                        :    0.00002  kg/N/s
Equivalence ratio φ         :  0.376
Mass flow compressor         : 54.042 kg/s
Fuel mass flow        :  0.022 kg/s
